In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd

import kraft

In [ ]:
name_id = pd.read_csv("cell-line-synon-names.txt", sep="\t", index_col=1)[
    "CELL_SAMPLE_ID"
]

id_aht = pd.read_csv("cell-line-best-names.txt", sep="\t", index_col=0)[
    "CELL_SAMPLE_NAME"
]

name_aht = {name: id_aht[id_] for name, id_ in name_id.items()}

len(name_aht)

In [ ]:
name_aht_ = pd.read_csv(
    "../../../project/ctd2/label/ice000.cols.cell_line_identity.2777.csv",
    usecols=(2, 3),
    index_col=0,
    squeeze=True,
).to_dict()

name_aht.update(name_aht_)

len(name_aht)

In [ ]:
for aht in tuple(name_aht.values()):

    name_aht[aht] = aht

len(name_aht)

In [ ]:
depmap_information_x_cell_line = pd.read_csv("sample_info.csv", index_col=1).T

depmap_information_x_cell_line

In [ ]:
for aht, names in depmap_information_x_cell_line.loc[
    ["DepMap_ID", "CCLE_Name", "Alias"]
].items():

    for name in names.dropna():

        aht_ = name_aht.get(name)

        if aht_ != aht:

            if aht_ is None:

                print("{}: {}".format(name, aht))

            else:

                print("{}: {} ==> {}".format(name, aht_, aht))

            name_aht[name] = aht

In [ ]:
name_aht = pd.Series(name_aht, name="Aht").sort_values()

name_aht.index.name = "Name"

name_aht.to_csv("cell_line_name_aht.tsv", sep="\t", header=True)

name_aht

In [ ]:
depmap_information_x_cell_line.loc["Is Fibroblast"] = (
    depmap_information_x_cell_line.loc["CCLE_Name"]
    .str.endswith("FIBROBLAST")
    .astype(int)
)

In [ ]:
information_x_cell_line = pd.concat((depmap_information_x_cell_line,))

information_x_cell_line.index.name = "Information"

information_x_cell_line.to_csv("information_x_cell_line.tsv", sep="\t")

information_x_cell_line

In [ ]:
dataframes = [information_x_cell_line.loc[["age"]]]

continuous_information_x_cell_line = pd.concat(dataframes)

continuous_information_x_cell_line.index.name = "Information"

continuous_information_x_cell_line.to_csv(
    "continuous_information_x_cell_line.tsv", sep="\t"
)

continuous_information_x_cell_line

In [ ]:
dataframes = [information_x_cell_line.loc[["Is Fibroblast"]]]

for index in (
    "sex",
    "sample_collection_site",
    "primary_or_metastasis",
    "primary_disease",
    "Subtype",
    "lineage",
    "lineage_subtype",
    "lineage_sub_subtype",
    "lineage_molecular_subtype",
):

    dataframe = kraft.binarize(information_x_cell_line.loc[index])

    dataframe.index = ("({}) {}".format(index, index_) for index_ in dataframe.index)

    dataframes.append(dataframe)

binary_information_x_cell_line = pd.concat(dataframes)

binary_information_x_cell_line.index.name = "Information"

binary_information_x_cell_line.to_csv("binary_information_x_cell_line.tsv", sep="\t")

binary_information_x_cell_line